<a href="https://colab.research.google.com/github/DEVjspf/prac_IA_gen_RAG/blob/main/prac_ia_gen_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ASISTENTE IA QUE RESPONDE TUS CONSULTAS DIRECTAMENTE DESDE DOCUMENTOS PDFs

## 🧩 Instalación de dependencias necesarias

In [1]:
!pip install pypdf gradio faiss-cpu

## 📦 Importación de librerías y módulos necesarios

In [2]:
import os
import shutil
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import faiss
import requests

## 🧠 Inicialización de estructuras para fragmentos y FAISS

In [3]:
# Diccionario para guardar los fragmentos con sus metadatos
fragmentos_indexados = []
index_faiss = None

## ⚙️ Configuración de constantes del sistema

In [4]:
if not os.path.exists("data/PDF"):
    os.makedirs("data/PDF")

In [5]:
RUTA_PDFS = "data/PDF"
MODELO_EMBEDDINGS = "all-MiniLM-L6-v2"
MODELO_LLM = "mistralai/Mixtral-8x7B-Instruct-v0.1"
os.environ["TOGETHER_API_KEY"] = "tgp_v1_PHdIPUdwUiD4Br7N3HjeZbxeNJuEdTRpW9LrfkFWZe8"

## 🧹 Función para limpiar datos anteriores y preparar carpeta de trabajo

In [6]:
def limpiar_datos_anteriores():

    # Limpiar la carpeta de los PDFs
    if os.path.exists(RUTA_PDFS):
        shutil.rmtree(RUTA_PDFS)
    os.makedirs(RUTA_PDFS, exist_ok=True)

    print("Datos anteriores eliminados completamente.")

## 📄 Función para extraer texto de un PDF página por página

In [7]:
def extraer_texto_pdf(ruta_pdf):
    lector = PdfReader(ruta_pdf)
    nombre_archivo = os.path.basename(ruta_pdf)
    texto_paginas = []
    for num_pagina, pagina in enumerate(lector.pages):
        texto = pagina.extract_text()
        if texto:
            texto_paginas.append({
                "texto": texto,
                "pagina": num_pagina + 1,
                "fuente": nombre_archivo
            })
    return texto_paginas

## 📥 Función para cargar y procesar todos los PDFs desde una carpeta

In [8]:
def cargar_documentos(ruta_carpeta):
    todos_fragmentos = []
    for archivo in os.listdir(ruta_carpeta):
        if archivo.endswith(".pdf"):
            ruta_pdf = os.path.join(ruta_carpeta, archivo)
            fragmentos = extraer_texto_pdf(ruta_pdf)
            todos_fragmentos.extend(fragmentos)
    return todos_fragmentos

## 🔍 Función para indexar fragmentos de texto con FAISS y generar embeddings

In [9]:
def indexar_fragmentos_faiss(fragmentos, modelo_embedding):
    global fragmentos_indexados, index_faiss

    if not fragmentos:
        print("⚠️ No hay fragmentos para indexar.")
        return

    print("🧠 Generando embeddings...")
    textos = [f["texto"] for f in fragmentos]
    embeddings = modelo_embedding.encode(textos, convert_to_numpy=True, normalize_embeddings=True)

    print("📦 Creando índice FAISS...")
    dim = embeddings.shape[1]
    index_faiss = faiss.IndexFlatIP(dim)
    index_faiss.add(embeddings)

    fragmentos_indexados = fragmentos
    print(f"✅ Se indexaron {len(fragmentos)} fragmentos.")

## 🧠 Función para buscar contexto relevante en el índice FAISS

In [10]:
def buscar_contexto_faiss(pregunta, modelo_embedding, k=3):
    if index_faiss is None:
        return "⚠️ No hay índice FAISS disponible.", []

    embedding_pregunta = modelo_embedding.encode([pregunta], convert_to_numpy=True, normalize_embeddings=True)
    distancias, indices = index_faiss.search(embedding_pregunta, k)

    chunks = []
    referencias = []
    for idx in indices[0]:
        frag = fragmentos_indexados[idx]
        ref = f"{frag['fuente']} - página {frag['pagina']}"
        referencias.append(ref)
        chunks.append(f"[{ref}]:\n{frag['texto']}")

    return "\n\n".join(chunks), referencias

## 🤖 Carga local del modelo LLM desde Hugging Face (no se usa directamente en la demo final)

In [11]:
def cargar_modelo_llm():
    tokenizer = AutoTokenizer.from_pretrained(
        MODELO_LLM,
        token=os.environ["HUGGINGFACE_TOKEN"],
        trust_remote_code=True
    )
    modelo = AutoModelForCausalLM.from_pretrained(
        MODELO_LLM,
        device_map="auto",
        token=os.environ["HUGGINGFACE_TOKEN"],
        trust_remote_code=True
    )
    generador = pipeline("text-generation", model=modelo, tokenizer=tokenizer)
    return generador

## ✉️ Función para generar respuesta vía API de Together utilizando contexto relevante

In [12]:
def responder_pregunta_con_together(pregunta):
    contexto, referencias = buscar_contexto_faiss(pregunta, modelo_embedding)

    if not contexto:
        return "No se encontró información relevante en los documentos cargados."

    prompt = f"""Contesta la siguiente pregunta usando solo la información proporcionada en el contexto.
Si no está en el contexto, responde que no tienes suficiente información. Responde en español. Por cada referencia
añade el documento y la página.

Contexto:
{contexto}

Pregunta: {pregunta}
Respuesta:"""

    response = requests.post(
        "https://api.together.xyz/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {os.environ['TOGETHER_API_KEY']}",
            "Content-Type": "application/json"
        },
        json={
            "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.3,
            "max_tokens": 512
        }
    )

    if response.status_code == 200:
        resultado = response.json()
        respuesta = resultado["choices"][0]["message"]["content"]
        referencias_unicas = list(set(referencias))
        citas = "\n\n📚 **Fuentes consultadas:**\n" + "\n".join(f"- {ref}" for ref in referencias_unicas)
        return respuesta + citas
    else:
        return f"Error al generar respuesta: {response.status_code} - {response.text}"

## 🧪 Función principal que ejecuta todo el flujo desde carga de PDFs hasta generación de respuesta

In [13]:
def interfaz_gradio(pdfs, pregunta):
    try:
        print("🧹 Limpiando datos anteriores...")
        limpiar_datos_anteriores()
        os.makedirs(RUTA_PDFS, exist_ok=True)

        for pdf in pdfs:
            if isinstance(pdf, str):
                nombre_archivo = os.path.basename(pdf)
                ruta_destino = os.path.join(RUTA_PDFS, nombre_archivo)

                # Evita copiar si ya está en el destino
                if os.path.abspath(pdf) != os.path.abspath(ruta_destino):
                    shutil.copy(pdf, ruta_destino)
            else:
                ruta_destino = os.path.join(RUTA_PDFS, pdf.name)
                with open(ruta_destino, "wb") as f:
                    f.write(pdf.read())

        print("✅ PDFs cargados.")

        fragmentos = cargar_documentos(RUTA_PDFS)
        print(f"🧠 Fragmentos cargados: {len(fragmentos)}")

        #indexar_fragmentos(fragmentos, cliente_chroma, modelo_embedding)
        indexar_fragmentos_faiss(fragmentos, modelo_embedding)
        print("📦 Embeddings indexados.")

        respuesta = responder_pregunta_con_together(pregunta)
        print("✅ Respuesta generada.")
        return respuesta
    except Exception as e:
        print("❌ Error detectado:", str(e))
        return f"Error: {str(e)}"

## 💻 Configuración de la interfaz gráfica con Gradio

In [14]:
demo = gr.Interface(
    fn=interfaz_gradio,
    inputs=[
        gr.File(file_types=[".pdf"], file_count="multiple", label="Sube tus archivos PDF"),
        gr.Textbox(label="Haz tu pregunta")
    ],
    outputs=gr.Textbox(label="Respuesta del sistema"),
    title="Sistema de Preguntas sobre PDFs"
)

modelo_embedding = SentenceTransformer(MODELO_EMBEDDINGS)

demo.launch()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ca9326e889599d35f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
